In [2]:
import fiftyone as fo
from PIL import Image
from detection import detect

In [3]:
name = "dataset-small"
dataset_dir = "/home/zenon/Documents/master-thesis/evaluation/dataset-small"

In [4]:
# The splits to load
splits = ["val"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset(name)
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
    )

classes = dataset.default_classes
predictions_view = dataset.view()

 100% |█████████████████| 401/401 [633.3ms elapsed, 0s remaining, 633.2 samples/s]      


In [5]:
# Do detections with model and save bounding boxes
with fo.ProgressBar() as pb:
    for sample in pb(predictions_view):
        image = Image.open(sample.filepath)
        w, h = image.size
        pred = detect(sample.filepath, 'yolo.onnx', 'resnet.onnx')

        detections = []
        for _, row in pred.iterrows():
            xmin, xmax = int(row['xmin']), int(row['xmax'])
            ymin, ymax = int(row['ymin']), int(row['ymax'])
            rel_box = [
                xmin / w, ymin / h, (xmax - xmin) / w, (ymax - ymin) / h
            ]
            detections.append(
                fo.Detection(label=classes[int(row['cls'])],
                             bounding_box=rel_box,
                             confidence=int(row['cls_conf'])))

        sample["yolo_resnet"] = fo.Detections(detections=detections)
        sample.save()

 100% |█████████████████| 401/401 [5.4m elapsed, 0s remaining, 1.4 samples/s]      


In [6]:
results = predictions_view.evaluate_detections(
    "yolo_resnet",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |█████████████████| 401/401 [1.2s elapsed, 0s remaining, 339.9 samples/s]         
Performing IoU sweep...
 100% |█████████████████| 401/401 [1.4s elapsed, 0s remaining, 288.5 samples/s]         


In [18]:
# Get the 10 most common classes in the dataset
counts = dataset.count_values("ground_truth.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top10)

print(results.mAP())

# Plot confusion matrix
matrix = results.plot_confusion_matrix(classes=classes)
matrix.show()

pr_curves = results.plot_pr_curves(classes=["Healthy", "Stressed"])
pr_curves.show()

              precision    recall  f1-score   support

     Healthy       0.80      0.81      0.81       430
    Stressed       0.77      0.72      0.75       315

   micro avg       0.79      0.77      0.78       745
   macro avg       0.79      0.77      0.78       745
weighted avg       0.79      0.77      0.78       745

0.6336217415940075


FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',
              'uid': '918e4315-b093-4d2b-8af4-789b5a5d5152',
              'x': array([0, 1, 2, 0, 1, 2, 0, 1, 2]),
              'y': array([0, 0, 0, 1, 1, 1, 2, 2, 2])},
             {'colorscale': [[0.0, 'rgb(255,245,235)'], [0.125,
                             'rgb(254,230,206)'], [0.25, 'rgb(253,208,162)'],
                             [0.375, 'rgb(253,174,107)'], [0.5, 'rgb(253,141,60)'],
                             [0.625, 'rgb(241,105,19)'], [0.75, 'rgb(217,72,1)'],
                             [0.875, 'rgb(166,54,3)'], [1.0, 'rgb(127,39,4)']],
              'hoverinfo': 'skip',
              'showscale': False,
              'type': 'heatmap',
              'uid': 'c2b76e66-be9d-4ca2-9060-d350e1d5e030',
              'z': array([[ 86,  68,   0],
                          [  0, 228,  87],
                          [348,   0,  82]]),
              'zmax': 348,
        

FigureWidget({
    'data': [{'customdata': array([99. , 99. , 99. , 99. , 99. , 99. , 99. , 99. , 99. , 99. , 99. , 98. ,
                                   98. , 98. , 97. , 97. , 97. , 97. , 97. , 96. , 96. , 95. , 94.2, 94. ,
                                   93.2, 93. , 92.9, 91.9, 91.2, 91. , 91. , 90.8, 89.8, 88.8, 88. , 87.9,
                                   87. , 86.9, 86. , 85.9, 85.8, 85. , 84.8, 84.6, 83.1, 81.9, 81.8, 81.1,
                                   80.1, 79.9, 79.7, 78.9, 78.6, 77.9, 77.7, 76.8, 76. , 75.6, 74.6, 74.2,
                                   73.3, 71.9, 70.7, 69.5, 68.3, 67.1, 66.2, 65.1, 64. , 62.9, 61.2, 60.6,
                                   59.7, 58.9, 52.6, 51.1, 49.4, 47.2, 46.4, 40.7, 30.5,  0. ,  0. ,  0. ,
                                    0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
                                    0. ,  0. ,  0. ,  0. ,  0. ]),
              'hovertemplate': ('<b>class: %{text}</b><br>reca

In [11]:
session = fo.launch_app(dataset, auto=False)
session.view = predictions_view
session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [20]:
session.plots.attach(matrix)